In [5]:
%%time
%%sh

# The name of our algorithm
# Makoto.Sano@Mack-the-Psych.com
algorithm_name=sagemaker-vdok3-bert

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  41.47kB
Step 1/27 : FROM nvidia/cuda:10.0-cudnn7-devel-ubuntu16.04
10.0-cudnn7-devel-ubuntu16.04: Pulling from nvidia/cuda
4f53fa4d2cf0: Pulling fs layer
6af7c939e38e: Pulling fs layer
903d0ffd64f6: Pulling fs layer
04feeed388b7: Pulling fs layer
4b61000175e2: Pulling fs layer
f8307f3573fe: Pulling fs layer
77a6c0aa636d: Pulling fs layer
38ebc0b38f6c: Pulling fs layer
4526afe95f0f: Pulling fs layer
db229a1287fc: Pulling fs layer
a6850e54a6be: Pulling fs layer
8e64b929e48f: Pulling fs layer
77a6c0aa636d: Waiting
38ebc0b38f6c: Waiting
4526afe95f0f: Waiting
db229a1287fc: Waiting
a6850e54a6be: Waiting
8e64b929e48f: Waiting
4b61000175e2: Waiting
f8307f3573fe: Waiting
04feeed388b7: Waiting
6af7c939e38e: Verifying Checksum
6af7c939e38e: Download complete
903d0ffd64f6: Download complete
04feeed388b7: Download complete
4b61000175e2: Verifying Checksum
4b61000175e2: Download complete
4f53fa4d2cf0: Verifying Checksum
4f53fa4d2cf0: Download c

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 89 ms, sys: 38.3 ms, total: 127 ms
Wall time: 18min 7s


In [6]:
# S3 prefix
# Makoto.Sano@Mack-the-Psych.com
prefix = 'vdok3_bert_trial'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [7]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [8]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [10]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Makoto.Sano@Mack-the-Psych.com
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-bert:latest'.format(account, region)
vdok3bert = sage.estimator.Estimator(image,
                       role, 1, 'ml.p2.xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
vdok3bert.fit(data_location)

2020-11-29 08:41:47 Starting - Starting the training job...
2020-11-29 08:41:49 Starting - Launching requested ML instances......
2020-11-29 08:42:57 Starting - Preparing the instances for training......
2020-11-29 08:44:10 Downloading - Downloading input data...
2020-11-29 08:44:22 Training - Downloading the training image.................................
2020-11-29 08:49:55 Failed - Training job failed
..

UnexpectedStatusException: Error for Training job sagemaker-vdok3-bert-2020-11-29-08-41-46-982: Failed. Reason: AlgorithmError: CannotStartContainerError. Please make sure the container can be run with 'docker run <image> train'. Please refer SageMaker documentation for details. It is possible that the Dockerfile's entrypoint is not properly defined, or missing permissions.

In [6]:
from sagemaker.predictor import csv_serializer
predictor = vdok3bert.deploy(1, 'ml.p2.xlarge', serializer=csv_serializer)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!

In [7]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

Score_Class
1
1
2
0



In [8]:
sess.delete_endpoint(predictor.endpoint)

In [9]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = vdok3bert.transformer(instance_count=1,
                               instance_type='ml.p2.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Using already existing model: sagemaker-vdok3-memn2n-2020-10-27-21-03-06-563


In [10]:
transformer.transform(data_location + '/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv', content_type='text/csv', split_type='Line')
transformer.wait()

s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out".format(transform_output_folder), '/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out')
with open('/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

....................................Starting the inference server with 4 workers.
2020/10/27 21:21:46 [crit] 12#0: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [27/Oct/2020:21:21:46 +0000] "GET /ping HTTP/1.1" 502 172 "-" "Go-http-client/1.1"
2020/10/27 21:21:46 [crit] 12#0: *3 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [27/Oct/2020:21:21:46 +0000] "GET /ping HTTP/1.1" 502 172 "-" "Go-http-client/1.1"
2020/10/27 21:21:46 [crit] 12#0: *5 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream,